In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx
from sklearn.ensemble import IsolationForest
from sklearn.preprocessing import StandardScaler
import seaborn as sns
from pyvis.network import Network
import community.community_louvain as community_louvain
from collections import Counter
import os
import logging
from pathlib import Path
import warnings
from itertools import combinations

warnings.filterwarnings('ignore')

class NetworkAnalysisTool:
    def __init__(self, base_path):
        """Initialize the Network Analysis Tool"""
        self.base_path = os.path.expanduser(base_path)
        self.output_path = os.path.join(self.base_path, 'output')
        os.makedirs(self.output_path, exist_ok=True)
        
        logging.basicConfig(
            filename=os.path.join(self.output_path, 'analysis.log'),
            level=logging.INFO,
            format='%(asctime)s - %(levelname)s - %(message)s'
        )
        
        self.load_data()
        self.create_graph()

    def load_data(self):
        """Load nodes and edges data"""
        try:
            self.nodes_df = pd.read_csv(os.path.join(self.base_path, 'ICS_OT Nodes.csv'))
            self.edges_df = pd.read_csv(os.path.join(self.base_path, 'ICS_OT Edges.csv'))
        except FileNotFoundError as e:
            raise FileNotFoundError(f"Error loading CSV files: {e}")
            
    def create_graph(self):
        """Create the network graph"""
        self.G = nx.Graph()
        
        for _, row in self.nodes_df.iterrows():
            self.G.add_node(row['Id'], label=row['Label'], shape=row.get('Shape', 'ellipse'))
            
        for _, row in self.edges_df.iterrows():
            self.G.add_edge(row['Source'], row['Target'])

    def analyze_network_structure(self):
        """Analyze basic network structure"""
        logging.info("Analyzing network structure")
        
        components = list(nx.connected_components(self.G))
        cycles = list(nx.cycle_basis(self.G))
        endpoints = [node for node, degree in dict(self.G.degree()).items() if degree == 1]
        
        self.centrality_measures = {
            'Degree_Centrality': nx.degree_centrality(self.G),
            'Betweenness_Centrality': nx.betweenness_centrality(self.G),
            'Closeness_Centrality': nx.closeness_centrality(self.G),
            'Eigenvector_Centrality': nx.eigenvector_centrality(self.G, max_iter=1000)
        }
        
        return {
            'components': components,
            'cycles': cycles,
            'endpoints': endpoints,
            'density': nx.density(self.G),
            'is_tree': nx.is_tree(self.G),
            'is_forest': nx.is_forest(self.G)
        }

    def analyze_bottlenecks(self):
        """Analyze network bottlenecks using spectral properties"""
        laplacian = nx.laplacian_matrix(self.G).todense()
        eigenvalues, eigenvectors = np.linalg.eigh(laplacian)
        
        n_components = 3
        bottleneck_scores = {}
        
        for node in self.G.nodes():
            idx = list(self.G.nodes()).index(node)
            score = 0
            for i in range(1, n_components + 1):
                weight = 1 / i
                score += weight * abs(eigenvectors[idx, i])
                
            degree = self.G.degree(node)
            if degree > 0:
                score = score / np.sqrt(degree)
                
            bottleneck_scores[node] = score
        
        threshold = np.percentile(list(bottleneck_scores.values()), 75)
        critical_bottlenecks = {
            node: score for node, score in bottleneck_scores.items() 
            if score > threshold
        }
        
        n_walks = 1000
        walk_length = 5
        flow_counts = {node: 0 for node in self.G.nodes()}
        
        for _ in range(n_walks):
            for start in self.G.nodes():
                current = start
                for _ in range(walk_length):
                    neighbors = list(self.G.neighbors(current))
                    if not neighbors:
                        break
                    current = np.random.choice(neighbors)
                    flow_counts[current] += 1
        
        max_flow = max(flow_counts.values())
        flow_centrality = {
            node: count/max_flow 
            for node, count in flow_counts.items()
        }
        
        bottleneck_analysis = {}
        for node in self.G.nodes():
            label = self.G.nodes[node]['label']
            bottleneck_analysis[label] = {
                'spectral_score': bottleneck_scores[node],
                'flow_centrality': flow_centrality[node],
                'is_critical': node in critical_bottlenecks,
                'degree': self.G.degree(node),
                'betweenness': self.centrality_measures['Betweenness_Centrality'][node]
            }
        
        return bottleneck_analysis

    def calculate_node_criticality(self):
        """Calculate node criticality based on network segmentation"""
        original_components = nx.number_connected_components(self.G)
        criticality = {}
        
        for node in self.G.nodes():
            G_temp = self.G.copy()
            G_temp.remove_node(node)
            new_components = nx.number_connected_components(G_temp)
            criticality[node] = new_components - original_components
            
        return criticality

    def calculate_security_metrics(self):
        """Calculate ICS/SCADA-specific security metrics"""
        control_systems = ['PLC 1', 'PLC 2', 'PLC 3', 'PLC 4', 'PLC 5']
        hmi_systems = ['HMI 1', 'HMI 2', 'HMI 3']
        field_devices = ['Sensor 1', 'Actuator 1'] + [f'Device {i}' for i in range(1, 12)]
        
        plc_nodes = [node for node, attr in self.G.nodes(data=True) if attr['label'] in control_systems]
        hmi_nodes = [node for node, attr in self.G.nodes(data=True) if attr['label'] in hmi_systems]
        field_nodes = [node for node, attr in self.G.nodes(data=True) if attr['label'] in field_devices]
        
        metrics = {}
        for node in self.G.nodes():
            metrics[node] = self._calculate_node_security_metrics(
                node, plc_nodes, hmi_nodes, field_nodes,
                control_systems, hmi_systems, field_devices
            )
            
        return metrics

    def _calculate_node_security_metrics(self, node, plc_nodes, hmi_nodes, field_nodes,
                                       control_systems, hmi_systems, field_devices):
        """Calculate security metrics for a single node"""
        node_label = self.G.nodes[node]['label']
        
        min_path_to_plc = float('inf')
        min_path_to_hmi = float('inf')
        
        for plc in plc_nodes:
            if nx.has_path(self.G, node, plc):
                min_path_to_plc = min(min_path_to_plc, nx.shortest_path_length(self.G, node, plc))
                
        for hmi in hmi_nodes:
            if nx.has_path(self.G, node, hmi):
                min_path_to_hmi = min(min_path_to_hmi, nx.shortest_path_length(self.G, node, hmi))
                
        return {
            'plc_connections': sum(1 for n in self.G.neighbors(node) if n in plc_nodes),
            'hmi_connections': sum(1 for n in self.G.neighbors(node) if n in hmi_nodes),
            'field_connections': sum(1 for n in self.G.neighbors(node) if n in field_nodes),
            'min_path_to_plc': min_path_to_plc,
            'min_path_to_hmi': min_path_to_hmi,
            'is_control_system': int(node_label in control_systems),
            'is_hmi': int(node_label in hmi_systems),
            'is_field_device': int(node_label in field_devices)
        }

    def detect_anomalies(self, security_metrics):
        """Enhanced anomaly detection with ICS-specific features"""
        features = pd.DataFrame()
        
        for measure, values in self.centrality_measures.items():
            features[measure] = pd.Series(values)
            
        for node in self.G.nodes():
            for metric, value in security_metrics[node].items():
                if metric not in features:
                    features[metric] = 0
                features.loc[node, metric] = float(value if value != float('inf') else 1000)
                
        features['plc_path_factor'] = features['min_path_to_plc'].apply(
            lambda x: 1/(x+1) if x < 1000 else 0)
        features['hmi_path_factor'] = features['min_path_to_hmi'].apply(
            lambda x: 1/(x+1) if x < 1000 else 0)
            
        features['exposure_score'] = (
            features['plc_connections'] * 3 +
            features['hmi_connections'] * 2 +
            features['field_connections'] +
            features['plc_path_factor'] * 2 +
            features['hmi_path_factor'] * 1.5
        )
        
        scaler = StandardScaler()
        features_scaled = scaler.fit_transform(features)
        
        iso_forest = IsolationForest(contamination=0.1, n_estimators=100, random_state=42)
        features['anomaly'] = iso_forest.fit_predict(features_scaled)
        features['anomaly_score'] = iso_forest.score_samples(features_scaled)
        
        min_score = features['anomaly_score'].min()
        max_score = features['anomaly_score'].max()
        features['risk_score'] = 100 * (features['anomaly_score'] - max_score) / (min_score - max_score)
        
        return features

    def calculate_spectral_metrics(self):
        """Calculate spectral metrics including spectral radius and Fiedler vector"""
        adjacency_matrix = nx.adjacency_matrix(self.G).todense()
        laplacian_matrix = nx.laplacian_matrix(self.G).todense()
        
        lap_eigenvalues, lap_eigenvectors = np.linalg.eigh(laplacian_matrix)
        
        spectral_radius = max(abs(lap_eigenvalues))
        
        idx = lap_eigenvalues.argsort()
        fiedler_value = lap_eigenvalues[idx[1]]
        fiedler_vector = lap_eigenvectors[:, idx[1]]
        
        node_labels = list(self.G.nodes())
        fiedler_components = list(zip(node_labels, fiedler_vector))
        fiedler_components.sort(key=lambda x: abs(x[1]), reverse=True)
        
        return {
            'spectral_radius': spectral_radius,
            'fiedler_value': fiedler_value,
            'fiedler_components': fiedler_components
        }

    def visualize_risk_levels(self, risk_scores):
        """Create network visualization with risk levels"""
        plt.figure(figsize=(15, 10))
        pos = nx.spring_layout(self.G, k=2, iterations=50)
        
        risk_colors = []
        node_sizes = []
        for node in self.G.nodes():
            risk_score = risk_scores.loc[node, 'risk_score']
            
            if risk_score >= 75:
                color = 'red'
                size = 3000
            elif risk_score >= 50:
                color = 'orange'
                size = 2500
            elif risk_score >= 25:
                color = 'yellow'
                size = 2000
            else:
                color = 'green'
                size = 1500
                
            risk_colors.append(color)
            node_sizes.append(size)
            
        nx.draw_networkx_edges(self.G, pos, edge_color='gray', alpha=0.5)
        nodes = nx.draw_networkx_nodes(self.G, pos,
                                     node_color=risk_colors,
                                     node_size=node_sizes)
                                     
        labels = nx.get_node_attributes(self.G, 'label')
        nx.draw_networkx_labels(self.G, pos, labels, font_size=8)
        
        legend_elements = [plt.Line2D([0], [0], marker='o', color='w',
                                    label=f'{level} Risk',
                                    markerfacecolor=color, markersize=10)
                          for level, color in [('Critical', 'red'),
                                             ('High', 'orange'),
                                             ('Moderate', 'yellow'),
                                             ('Low', 'green')]]
                                             
        plt.legend(handles=legend_elements, loc='upper left',
                  title='Risk Levels', bbox_to_anchor=(1, 1))
                  
        plt.title('Network Risk Level Visualization')
        plt.axis('off')
        plt.tight_layout()
        
        plt.savefig(os.path.join(self.output_path, 'risk_level_visualization.png'),
                    bbox_inches='tight', dpi=300)
        plt.close()

    def visualize_bottlenecks(self, bottleneck_analysis):
        """Create visualization of network bottlenecks"""
        fig, ax = plt.subplots(figsize=(15, 10))
        pos = nx.spring_layout(self.G, k=2, iterations=50)
        
        scores = [bottleneck_analysis[self.G.nodes[node]['label']]['spectral_score'] 
                 for node in self.G.nodes()]
    
        norm = plt.Normalize(vmin=min(scores), vmax=max(scores))
        node_colors = plt.cm.YlOrRd(norm(scores))
    
        flow_centrality = [bottleneck_analysis[self.G.nodes[node]['label']]['flow_centrality'] * 3000 
                          for node in self.G.nodes()]
    
        nx.draw_networkx_edges(self.G, pos, edge_color='gray', alpha=0.5, ax=ax)
        nodes = nx.draw_networkx_nodes(self.G, pos, 
                                     node_color=node_colors,
                                     node_size=flow_centrality,
                                     ax=ax)
    
        labels = nx.get_node_attributes(self.G, 'label')
        nx.draw_networkx_labels(self.G, pos, labels, font_size=8, ax=ax)
    
        sm = plt.cm.ScalarMappable(cmap=plt.cm.YlOrRd, norm=norm)
        plt.colorbar(sm, ax=ax, label='Bottleneck Score')
    
        ax.set_title('Network Bottleneck Analysis')
        ax.axis('off')
    
        plt.tight_layout()
        plt.savefig(os.path.join(self.output_path, 'bottleneck_analysis.png'),
                    bbox_inches='tight', dpi=300)
        plt.close()

    def _analyze_zone_isolation(self, zone, security_metrics):
        """Analyze potential zone isolation violations"""
        violations = []
        
        zone_nodes = {
            'Control': [n for n in self.G.nodes() if self.G.nodes[n]['label'].startswith('PLC')],
            'Operations': [n for n in self.G.nodes() if self.G.nodes[n]['label'].startswith('HMI')],
            'Field': [n for n in self.G.nodes() if 'Device' in self.G.nodes[n]['label'] or 
                     any(x in self.G.nodes[n]['label'] for x in ['Sensor', 'Actuator'])]
        }
        
        for node in zone_nodes.get(zone, []):
            neighbors = list(self.G.neighbors(node))
            for neighbor in neighbors:
                if not any(neighbor in nodes for nodes in zone_nodes.values()):
                    violations.append((node, neighbor))
                    
        return violations

    def _find_exposed_paths(self, asset):
        """Find potentially exposed paths to critical assets"""
        target_node = None
        for node in self.G.nodes():
            if self.G.nodes[node]['label'] == asset:
                target_node = node
                break
                
        if not target_node:
            return []
            
        paths = []
        for node in self.G.nodes():
            if node != target_node and nx.has_path(self.G, node, target_node):
                path = nx.shortest_path(self.G, node, target_node)
                if len(path) <= 3:  # Consider paths of length 3 or less as exposed
                    paths.append(path)
                    
        return paths

    def generate_executive_report(self, results):
        """Generate security-focused report with improved formatting"""
        bottleneck_analysis = results['bottleneck_analysis']
        risk_scores = results['risk_scores']
        security_metrics = results['security_metrics']
        
        # Executive Summary section
        high_risk_nodes = len(risk_scores[risk_scores['risk_score'] >= 75])
        critical_bottlenecks = sum(1 for m in bottleneck_analysis.values() if m['is_critical'])
        exposed_assets = sum(1 for m in security_metrics.values() 
                           if m['min_path_to_plc'] <= 2 or m['min_path_to_hmi'] <= 2)

        report = f"""# ICS/OT Network Security Analysis Report

## Executive Summary

CRITICAL FINDINGS:
• {high_risk_nodes} nodes identified as high-risk requiring immediate attention
• {critical_bottlenecks} critical network bottlenecks that could impact operations
• {exposed_assets} potentially exposed critical assets

## Network Vulnerability Assessment

### High-Risk Nodes
```
Node         | Risk Score | PLC Conn | HMI Conn | Field Conn | Exposure
-------------|------------|----------|----------|------------|----------"""

        high_risk = risk_scores[risk_scores['risk_score'] >= 75].sort_values('risk_score', ascending=False)
        for node in high_risk.index[:5]:
            node_label = self.G.nodes[node]['label']
            risk_score = risk_scores.loc[node, 'risk_score']
            metrics = security_metrics[node]
            report += f"\n{node_label:<12} | {risk_score:^10.1f} | {metrics['plc_connections']:^8} | {metrics['hmi_connections']:^8} | {metrics['field_connections']:^10} | {risk_scores.loc[node, 'exposure_score']:^8.2f}"

        report += "\n```\n"

        # Bottleneck Analysis section
        report += """
## Network Bottleneck Analysis

Critical traffic concentration points ranked by impact:

```
Node         | Impact Score | Traffic Load | Components | Risk Level
-------------|-------------|--------------|------------|------------"""

        critical_points = sorted(
            ((node, metrics) for node, metrics in bottleneck_analysis.items() if metrics['is_critical']),
            key=lambda x: x[1]['spectral_score'],
            reverse=True
        )
        
        for node, metrics in critical_points[:5]:
            impact = metrics['spectral_score']
            risk_level = "HIGH" if impact > 0.7 else "MODERATE"
            report += f"\n{node:<12} | {impact:^11.3f} | {metrics['flow_centrality']:^11.3f} | {metrics['degree']:^10} | {risk_level:<10}"

        report += "\n```"

        # Zone Analysis section
        report += """

## Security Zone Analysis

Zone isolation status:

```
Zone         | Violations | Risk Level | Recommended Action
-------------|------------|------------|-------------------"""
        
        for zone in ['Control', 'Operations', 'Field']:
            violations = self._analyze_zone_isolation(zone, security_metrics)
            risk_level = "HIGH" if len(violations) > 3 else "MODERATE"
            action = "IMMEDIATE REVIEW" if len(violations) > 3 else "Monitor and Review"
            report += f"\n{zone:<12} | {len(violations):^10} | {risk_level:<10} | {action}"

        report += "\n```"

        # Attack Path Analysis section
        report += """

## Attack Path Analysis

Critical asset exposure analysis:

```
Asset        | Attack Paths | Min Path Length | Risk Level
-------------|-------------|-----------------|------------"""
        
        critical_assets = ['Crown Jewel', 'SCADA Server', 'HMI 1', 'HMI 2', 'HMI 3']
        for asset in critical_assets:
            paths = self._find_exposed_paths(asset)
            if paths:
                risk_level = "HIGH" if len(paths) > 5 else "MODERATE"
                report += f"\n{asset:<12} | {len(paths):^12} | {min(len(p) for p in paths):^15} | {risk_level:<10}"

        report += "\n```"

        # Technical Details section
        report += """

## Appendix: Technical Details

```
Metric               | Value
--------------------|-------"""
        
        report += f"""
Total Nodes         | {self.G.number_of_nodes()}
Network Diameter    | {nx.diameter(self.G)}
Average Path Length | {nx.average_shortest_path_length(self.G):.2f}
Graph Density      | {nx.density(self.G):.3f}
Spectral Radius    | {self.calculate_spectral_metrics()['spectral_radius']:.3f}
Fiedler Value      | {self.calculate_spectral_metrics()['fiedler_value']:.3f}
```
Top Fiedler Components:
"""
        
        for node, value in self.calculate_spectral_metrics()['fiedler_components'][:5]:
            report += f"{node:<12} | {value:.3f}\n"

        report += "\n```"

        return report

    def run_analysis(self):
        """Run complete network analysis"""
        # Basic analysis
        structure_analysis = self.analyze_network_structure()
        criticality_scores = self.calculate_node_criticality()
        security_metrics = self.calculate_security_metrics()
        risk_scores = self.detect_anomalies(security_metrics)
        spectral_metrics = self.calculate_spectral_metrics()
        
        # Bottleneck analysis
        bottleneck_analysis = self.analyze_bottlenecks()
        
        results = {
            'structure_analysis': structure_analysis,
            'criticality_scores': criticality_scores,
            'security_metrics': security_metrics,
            'risk_scores': risk_scores,
            'spectral_metrics': spectral_metrics,
            'bottleneck_analysis': bottleneck_analysis
        }
        
        # Generate visualizations and reports
        self.visualize_risk_levels(risk_scores)
        self.visualize_bottlenecks(bottleneck_analysis)
        report = self.generate_executive_report(results)
        
        # Save the report
        report_path = os.path.join(self.output_path, 'security_analysis_report.md')
        with open(report_path, 'w', encoding='utf-8') as f:
            f.write(report)
            
        print(f"\nAnalysis complete. Security report saved to: {report_path}")
        return results
    
if __name__ == "__main__":
    base_path = r"C:\Users\Service Casket\Desktop\Network Analysis Tool"
    tool = NetworkAnalysisTool(base_path)
    results = tool.run_analysis()



Analysis complete. Security report saved to: C:\Users\Service Casket\Desktop\Network Analysis Tool\output\security_analysis_report.md
